# Problem 3 Naive Bayes

## 3.1 implement a Naive Bayes classifier

In [1]:
import sys
import itertools
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
import time
from copy import deepcopy

In [2]:
class Model:
    def __init__(self, wordlist):
        self.wordlist = wordlist

    def count_labels(self, data):
        """
        Count the number of positive labels and negative labels.
        Returns (a tuple or a numpy array of two elements):
            * negative_count: a non-negative integer, which represents the number of negative labels (non-spam emails);
            * positive_count: a non-negative integer, which represents the number of positive labels (spam emails).
        """
        # TODO
        positive_count = sum([y for y, x in data])
        negative_count = len(data) - positive_count
        return np.array([negative_count, positive_count])

    def count_words(self, wordlist, data):
        """
        Count the number of times that each word appears in emails under a given label.
        Returns (a numpy array):
            * word_counts: a numpy array with shape (2, L), where L is the length of $wordlist,
                - word_counts[0, i] represents the number of times that word $wordlist[i] appears in non-spam (negative) emails, and
                - word_counts[1, i] represents the number of times that word $wordlist[i] appears in spam (positive) emails.
        """
        # TODO
        word_vector = np.zeros((2, len(wordlist)))
        
        for i in range(len(wordlist)):
            for j in range(len(data)):
                if wordlist[i] in data[j][1]:
                    if data[j][0] == 1:
                        word_vector[1,i] += 1
                    else:
                        word_vector[0,i] += 1
        return word_vector

    def calculate_probability(self, label_counts, word_counts):
        """
        Calculate the probabilities, both the prior and likelihood.
        Returns (a pair of numpy array):
            * prior_probs: a numpy array with shape (2, ), only two elements, where
                - prior_probs[0] is the prior probability of negative labels, and
                - prior_probs[1] is the prior probability of positive labels.
            * likelihood_probs: a numpy array with shape (2, L), where L is the length of the word list,
                - likelihood_probs[0, i] represents the likelihood probability of the $i-th word in the word list, given that the email is non-spam (negative), and
                - likelihood_probs[1, i] represents the likelihood probability of the $i-th word in the word list, given that the email is spam (positive).
        """
        # TODO
        prior_probs = np.zeros((2,))
        likelihood_probs = np.zeros((2,len(self.wordlist)))
        
        negative_probs = label_counts[0]/ (label_counts[0] + label_counts[1])
        positive_probs = 1 - negative_probs
        
        prior_probs[0] = negative_probs
        prior_probs[1] = positive_probs
        
        
        nonspam_probs = (word_counts[0]+1)/(label_counts[0]+2)
        spam_probs = (word_counts[1]+1)/(label_counts[1]+2)
        likelihood_probs[0] = nonspam_probs
        likelihood_probs[1] = spam_probs
        
        return (prior_probs, likelihood_probs)

    def fit(self, data):
        label_counts = self.count_labels(data)
        word_counts = self.count_words(self.wordlist, data)

        self.prior_probs, self.likelihood_probs = self.calculate_probability(label_counts, word_counts)

        # TO AVOID NUMBER OVERFLOW here we use log probability instead.
        self.log_prior_probs = np.log(self.prior_probs)
        self.log_likelihood_probs = np.dstack([np.log(1 - self.likelihood_probs), np.log(self.likelihood_probs)])

    def predict(self, x):
        """
        Predict whether email $x is a spam or not.
        Returns:
            * y: a boolean value indicating whether $x is a spam or not.
        """        
        spam_prob = self.log_prior_probs[1]
        nonspam_prob = self.log_prior_probs[0]
        
        for i in range(len(self.wordlist)):
            if self.wordlist[i] in x:
                exists = 1
            else:
                exists = 0
            
            nonspam_prob += self.log_likelihood_probs[0][i][exists]
            
            spam_prob += self.log_likelihood_probs[1][i][exists]
            
        if spam_prob > nonspam_prob:
            y = 1
        else:
            y =0
        
        return y

In [3]:
def read_data(filename):
    """
    Read the dataset from the file given by name $filename.
    The returned object should be a list of pairs of data. In each pair: the first element is 1 (for spam emails) 
    or 0 (for non-spam emails), the second element is a list of words in the email.
    The returned list: 
        [
            (1 , ['a', 'b', 'c']),
            (0, ['d', 'e', 'f']),
            ...
        ]
    """
    with open(filename, 'r') as f:
        return [(int(y), x) for y, *x in [line.strip().split() for line in f]]

def split_train(original_train_data, size=4000):
    return original_train_data[:size], original_train_data[size:]


def create_wordlist(original_train_data, threshold=26):
    """
    Create a word list from the original training set.
    Only get a word if it appears in at least $threshold emails.
    Returns:
        * a python list containing all the words that occur in at least $threshold emails.
    """
    vocab_dict = {}
    for each_email in original_train_data:
        for word in set(each_email[1]):
            vocab_dict[word] = vocab_dict.get(word, 0) + 1

    wordlist = []
    for key, val in vocab_dict.items():
        if val >= threshold:
            wordlist.append(key)
    return wordlist

In [4]:
# threshold to determine whether to include a word in the dictionary/wordlist.
# ie. only words with frequency higher than threshold are included
THRESHOLD = 26

In [5]:
original_train_data = read_data('spam_train.txt')

# further split the data into a training set and a validation set
train_data, val_data = split_train(original_train_data)

# Create the word list.
wordlist = create_wordlist(original_train_data, 26)
print("Total # of words:", len(wordlist))

# fit the model using train_data
model = Model(wordlist)
model.fit(train_data)


# TODO
# calculate the error rate on val_data (when threshold=26)
# print out the error rate
error_count = sum([y != model.predict(x) for y, x in val_data])
error_percentage = error_count / len(val_data) * 100

print("Validation error, # = {:>4d}, % = {:>8.4f}%.".format(error_count, error_percentage)) 

Total # of words: 3048
Validation error, # =   61, % =   6.1000%.


## 3.2 try different thresholds, find the optimal threshold (which gives minimum validation error), print out the test error at the optimal threshold

In [6]:
def compute_error_rate(model, data):
    
    error_count = sum([y != model.predict(x) for y, x in data])
    return 100.0 * error_count / len(data)

In [7]:
thresholds = list(range(1, 35))
train_error = []
val_error = []
test_error = []
original_train_data = read_data('spam_train.txt')
train_data, val_data = split_train(original_train_data)
test_data = read_data('spam_test.txt')

In [8]:
for th in thresholds:
    print('With threshold {}....'.format(th))
    time1 = time.time()

    # vocabulary selection
    wordlist = create_wordlist(original_train_data, th)

    # fit model using the wordlist
    model = Model(wordlist)
    model.fit(train_data)

    # compute classification error rates
    err_train = compute_error_rate(model, train_data)
    err_val = compute_error_rate(model, val_data)
    err_test = compute_error_rate(model, test_data)

    # store results for plotting
    train_error.append(err_train)
    val_error.append(err_val)
    test_error.append(err_test)

    time2 = time.time()
    print("train:{} val:{} test:{} len(V)={}".format(err_train, err_val, err_test, len(wordlist)))
    print('time: {}'.format(time2 - time1))

With threshold 1....


KeyboardInterrupt: 

In [ ]:
# plot the training and validation error rate vs. the thresholds
# choose the threshold with the minimal validation error rate and report the corresponding test error rate

# TODO
plt.figure()
plt.plot(range(1, 35),train_error,label="training error")
plt.plot(range(1, 35),val_error,label="validation error")
plt.xlabel('thresholds')
plt.ylabel('error rate')
plt.title('The training and validation error rate vs. the thresholds')
plt.legend()
plt.show()

min_value = min(test_error)
opt = test_error.index(min_value)
print('Best performance at validated threshold {} with test error rate {}.'.format(opt, test_error[opt]))